## Add hexapod motions to the mount plots

Craig Lage  04-Jun-25

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from lsst.daf.butler import Butler, DimensionRecord
from lsst.summit.utils.efdUtils import makeEfdClient
from lsst.summit.utils.simonyi.mountAnalysis import calculateMountErrors, plotMountErrors
from lsst.summit.utils.butlerUtils import getExpRecordFromDataId
from astropy.time import Time, TimeDelta
import yaml, copy
from dataclasses import dataclass
from pandas import DataFrame

In [ ]:
client = makeEfdClient()
butler = Butler('/repo/embargo', collections=['LSSTCam/raw/all', 
                                            'LSSTCam/calib/unbounded', 'LSSTCam/runs/nightlyValidation'])

In [ ]:
expId = 2025071800254
#expId = 2025071800445
#expId = 2025080600060
dataId = {'exposure':expId, 'instrument':'LSSTCam'}
expRecord = getExpRecordFromDataId(butler, dataId)

In [ ]:
(mountErrors, mountData) = calculateMountErrors(expRecord, client)
print(mountErrors.camHexRms, mountErrors.m2HexRms)

In [ ]:

lut_path_update = "/home/c/cslage/u/Hexapods/LUTs/_init.yaml"

with open(lut_path_update, "r") as yaml_file:
    lut_data = yaml.safe_load(yaml_file)


In [ ]:
lut_data

In [ ]:

@dataclass
class MountData:
    begin: Time
    end: Time
    azimuthData: DataFrame
    elevationData: DataFrame
    rotationData: DataFrame
    rotationTorques: DataFrame
    camhexData: DataFrame
    m2hexData: DataFrame
    includedPrePadding: float
    includedPostPadding: float
    expRecord: DimensionRecord | None


In [ ]:
def calculateHexRms(mountData: MountData) -> tuple[float, float]:
    """Calculate the image impact of hexapod motions.

    Parameters
    ----------
    mountData : MountData
        The EFD data associated with the exposure

    Returns
    -------
    tuple[float, float]
        The image motions associated with the CamHex and M2Hex motions.
    """
    # The below image motion coefficients were calculated
    # with a Batoid simulation by Josh Meyers
    camHexXY = 1.00  # microns(image) / micron(hexapod)
    camHexUV = 4.92  # microns(image) / arcsecond(hexapod)
    m2HexXY = 1.13  # microns(image) / micron(hexapod)
    m2HexUV = 37.26  # microns(image) / arcsecond(hexapod)

    # Convert these to image impact in arcseconds
    # The 10.0 is microns / pixel
    pixelScale = 0.2  # arcseconds / pixel - find this elsewhere?
    camHexXY = camHexXY / 10.0 * pixelScale  # arcseconds(image) / micron(hexapod)
    camHexUV = camHexUV / 10.0 * pixelScale  # arcseconds(image) / arcsecond(hexapod)
    camCoefs = [camHexXY, camHexXY, 0, camHexUV, camHexUV, 0]
    m2HexXY = m2HexXY / 10.0 * pixelScale  # arcseconds(image) / micron(hexapod)
    m2HexUV = m2HexUV / 10.0 * pixelScale  # arcseconds(image) / arcsecond(hexapod)
    m2Coefs = [m2HexXY, m2HexXY, 0, m2HexUV, m2HexUV, 0]

    els = mountData.elevationData['actualPosition'].values
    elTimes = mountData.elevationData['timestamp'].values
    camHexMs = 0.0
    for i in [0, 1, 3, 4]:
        lut_coeffs = lut_data['camera_config']['elevation_coeffs'][i]
        camhex = copy.deepcopy(mountData.camhexData[f"position{i}"])
        camhex -= np.median(camhex)
        camTimes = np.asarray(mountData.camhexData['private_kafkaStamp'])
        # Below are the LUT values.  Only elevation at this point
        dz = np.polyval(lut_coeffs[::-1], els)
        dz -= np.median(dz)
        # Need to align the timestamps, then subtract LUT
        dzInterp = np.interp(camTimes, elTimes, dz)
        camhex -= dzInterp
        camhex *= camCoefs[i]
        camHexMs += np.mean(camhex * camhex)
    camHexRms = np.sqrt(camHexMs)  # in arcseconds image impact

    m2HexMs = 0.0
    for i in [0, 1, 3, 4]:
        lut_coeffs = lut_data['m2_config']['elevation_coeffs'][i]
        m2hex = copy.deepcopy(mountData.m2hexData[f"position{i}"])
        m2hex -= np.median(m2hex)
        m2Times = np.asarray(mountData.m2hexData['private_kafkaStamp'])
        # Below are the LUT values.  Only elevation at this point
        dz = np.polyval(lut_coeffs[::-1], els)
        dz -= np.median(dz)
        # Need to align the timestamps, then subtract LUT
        dzInterp = np.interp(m2Times, elTimes, dz)
        m2hex -= dzInterp
        m2hex *= m2Coefs[i]
        m2HexMs += np.mean(m2hex * m2hex)
    m2HexRms = np.sqrt(m2HexMs)  # in arcseconds image impact
    return (float(camHexRms), float(m2HexRms))

In [ ]:
def calculateHexRmsError(mountData: MountData) -> tuple[float, float]:
    """Calculate the image impact of hexapod motions.

    Parameters
    ----------
    mountData : MountData
        The EFD data associated with the exposure

    Returns
    -------
    tuple[float, float]
        The image motions associated with the CamHex and M2Hex motions.
    """
    # The below image motion coefficients were calculated
    # with a Batoid simulation by Josh Meyers
    camHexXY = 1.00  # microns(image) / micron(hexapod)
    camHexUV = 4.92  # microns(image) / arcsecond(hexapod)
    m2HexXY = 1.13  # microns(image) / micron(hexapod)
    m2HexUV = 37.26  # microns(image) / arcsecond(hexapod)

    # Convert these to image impact in arcseconds
    # The 10.0 is microns / pixel
    pixelScale = 0.2  # arcseconds / pixel - find this elsewhere?
    camHexXY = camHexXY / 10.0 * pixelScale  # arcseconds(image) / micron(hexapod)
    camHexUV = camHexUV / 10.0 * pixelScale  # arcseconds(image) / arcsecond(hexapod)
    camCoefs = [camHexXY, camHexXY, 0, camHexUV, camHexUV, 0]
    m2HexXY = m2HexXY / 10.0 * pixelScale  # arcseconds(image) / micron(hexapod)
    m2HexUV = m2HexUV / 10.0 * pixelScale  # arcseconds(image) / arcsecond(hexapod)
    m2Coefs = [m2HexXY, m2HexXY, 0, m2HexUV, m2HexUV, 0]

    camHexMs = 0.0
    for i in [0, 1, 2, 3, 4]:
        camhex = copy.deepcopy(mountData.camhexData[f"error{i}"])
        camhex *= camCoefs[i]
        camHexMs += np.mean(camhex * camhex)
    camHexRms = np.sqrt(camHexMs)  # in arcseconds image impact

    m2HexMs = 0.0
    for i in [0, 1, 2, 3, 4]:
        m2hex = copy.deepcopy(mountData.m2hexData[f"error{i}"])
        m2hex *= m2Coefs[i]
        m2HexMs += np.mean(m2hex * m2hex)
    m2HexRms = np.sqrt(m2HexMs)  # in arcseconds image impact
    return (float(camHexRms), float(m2HexRms))

In [ ]:
print(mountErrors.camHexRms, mountErrors.m2HexRms)

In [ ]:
# About 10X smaller
newCamhexRms, newM2hexRms = calculateHexRmsError(mountData)
print(newCamhexRms, newM2hexRms)

In [ ]:
fig, axs = plt.subplots(5, 2, figsize = (16, 8))
plt.subplots_adjust(hspace=1.0, wspace=0.5)
els = mountData.elevationData['actualPosition'].values
elTimes = mountData.elevationData['timestamp'].values
names = ['X', 'Y', 'Z', 'U', 'V']
j = 0
for m, i in enumerate([0, 1, 2, 3, 4]):
    lut_coeffs = lut_data['camera_config']['elevation_coeffs'][i]
    camhex = copy.deepcopy(mountData.camhexData[f"position{i}"])
    camhex -= np.median(camhex)
    camTimes = np.asarray(mountData.camhexData['private_kafkaStamp'])
    # Below are the LUT values.  Only elevation at this point
    dz = np.polyval(lut_coeffs[::-1], els)
    dz -= np.median(dz)
    dzInterp = np.interp(camTimes, elTimes, dz)

    if i in [3,4]:
        camhex *= 3600
        dzInterp *= 3600
    # Need to align the timestamps, then subtract LUT
    axs[m][j].plot(camTimes, camhex, label="CamHex dz Measured - median subtracted")
    axs[m][j].plot(camTimes, dzInterp, label="CamHex dz LUT - median subtracted")
    axs[m][j].set_title(f"Cam {names[i]}, Meas(blue) vs LUT(yellow) {expId}")
    axs[m][j].set_ylabel("Hex (mic, arcsec)")
    axs[m][j].set_xlabel("Time") 
    #axs[m][j].legend()

    
j = 1
m2HexMs = 0.0
for m, i in enumerate([0, 1, 2, 3, 4]):
    lut_coeffs = lut_data['m2_config']['elevation_coeffs'][i]
    m2hex = copy.deepcopy(mountData.m2hexData[f"position{i}"])
    m2hex -= np.median(m2hex)
    m2Times = np.asarray(mountData.m2hexData['private_kafkaStamp'])
    # Below are the LUT values.  Only elevation at this point
    dz = np.polyval(lut_coeffs[::-1], els)
    dz -= np.median(dz)
    # Need to align the timestamps, then subtract LUT
    dzInterp = np.interp(m2Times, elTimes, dz)
    if i in [3,4]:
        m2hex *= 3600
        dzInterp *= 3600
    axs[m][j].plot(camTimes, camhex, label="M2Hex dz Measured - median subtracted")
    axs[m][j].plot(camTimes, dzInterp, label="M2Hex dz LUT - median subtracted")
    axs[m][j].set_title(f"M2 {names[i]}, Meas(blue) vs LUT(yellow) {expId}")
    axs[m][j].set_ylabel("Hex (mic, arcsec)")
    axs[m][j].set_xlabel("Time") 
    #axs[m][j].legend()

saveFilename = f"/home/c/cslage/u/MTMount/mount_plots/Hexapod_LUT_Subtraction_{expId}.png"
plt.savefig(saveFilename)

In [ ]:
lut_coeffs = lut_data['camera_config']['elevation_coeffs'][1]


In [ ]:
camhex = copy.deepcopy(mountData.camhexData[f"position{1}"])
camhex -= np.median(camhex)
print(len(camhex))

els = np.asarray(mountData.elevationData['actualPosition'])
elTimes = np.asarray(mountData.elevationData['timestamp'])
camTimes = np.asarray(mountData.camhexData['private_kafkaStamp'])
dz = np.polyval(lut_coeffs[::-1], els)
dz -= np.median(dz)


In [ ]:
camTimes[0]

In [ ]:
elTimes[0]

In [ ]:
dzInterp = np.interp(camTimes, elTimes, dz)
print(len(dzInterp))

In [ ]:
plt.plot(camTimes, camhex, label="CamHex dz Measured - median subtracted")
plt.plot(camTimes, dzInterp, label="CamHex dz LUT - median subtracted")
plt.title(f"DM-52072 - subtracting LUT {expId}")
plt.ylabel("Hexapod motion (microns)")
plt.xlabel("Time") 
plt.legend()
saveFilename = f"/home/c/cslage/u/MTMount/mount_plots/Hexapod_LUT_Subtraction_{expId}.png"
plt.savefig(saveFilename)

In [ ]:
plt.plot(camTimes, camhex-dzInterp)

In [ ]:
mountData.camhexData.columns

In [ ]:
# The below image motion coefficients were calculated
# with a Batoid simulation by Josh Meyers
camHexXY = 1.00  # microns(image) / micron(hexapod)
camHexUV = 4.92  # microns(image) / arcsecond(hexapod)
m2HexXY = 1.13  # microns(image) / micron(hexapod)
m2HexUV = 37.26  # microns(image) / arcsecond(hexapod)

pixelScale = 0.2  # arcseconds / pixel - find this elsewhere?
camHexXY = camHexXY / 10.0 * pixelScale  # arcseconds(image) / micron(hexapod)
camHexUV = camHexUV / 10.0 * pixelScale  # arcseconds(image) / arcsecond(hexapod)
camCoefs = [camHexXY, camHexXY, 0, camHexUV, camHexUV, 0]
m2HexXY = m2HexXY / 10.0 * pixelScale  # arcseconds(image) / micron(hexapod)
m2HexUV = m2HexUV / 10.0 * pixelScale  # arcseconds(image) / arcsecond(hexapod)
m2Coefs = [m2HexXY, m2HexXY, 0, m2HexUV, m2HexUV, 0]


In [ ]:
(mountErrors, mountData) = calculateMountErrors(expRecord, client)



lut_coeffs = lut_data['m2_config']['elevation_coeffs'][0]
m2hex = copy.deepcopy(mountData.m2hexData[f"position{0}"])
m2hex -= np.median(m2hex)
print(len(m2hex))

els = np.asarray(mountData.elevationData['actualPosition'])
elTimes = np.asarray(mountData.elevationData['timestamp'])
m2Times = np.asarray(mountData.m2hexData['private_kafkaStamp'])
dz = np.polyval(lut_coeffs[::-1], els)
dz -= np.median(dz)

dzInterp = np.interp(m2Times, elTimes, dz)
print(len(dzInterp))

plt.plot(m2Times, m2hex, label="M2Hex dz Measured - median subtracted")
plt.plot(m2Times, dzInterp, label="M2Hex dz LUT - median subtracted")
plt.title(f"DM-52072 - subtracting LUT {expId}")
plt.ylabel("Hexapod motion (microns)")
plt.xlabel("Time") 
plt.legend()

m2hex -= dzInterp
m2hex *= m2Coefs[0]
m2HexMs = np.mean(m2hex * m2hex)
m2HexRms = np.sqrt(m2HexMs)  # in arcseconds image impact
plt.text((expRecord.timespan.begin + TimeDelta(1, format='sec')).unix_tai, -0.10, f"RMS error = {m2HexRms:.2f} arcsec")
saveFilename = f"/home/c/cslage/u/MTMount/mount_plots/Hexapod_LUT_Subtraction_{expId}.png"
plt.savefig(saveFilename)


In [ ]:
(mountErrors, mountData) = calculateMountErrors(expRecord, client)



fig, axs = plt.subplots(1,2, figsize=(10,5))
plt.suptitle(f"DM-52072 - Hexapod Errors {expId}")
plt.subplots_adjust(wspace=0.5)
mountData.m2hexData['position0'].plot(ax=axs[0], label="Position0")
mountData.m2hexData['demand0'].plot(ax=axs[0], label="Demand0")
axs[0].set_ylabel("Hexapod motion (microns)")
axs[0].set_xlabel("Time") 
axs[0].legend()

mountData.m2hexData['error0'].plot(ax=axs[1], label="Error0")
axs[1].set_ylabel("Hexapod motion (microns)")
axs[1].set_xlabel("Time") 
axs[1].legend()

err = mountData.m2hexData['error0'].values
err *= m2Coefs[0]
m2HexMs = np.mean(err * err)
m2HexRms = np.sqrt(m2HexMs)  # in arcseconds image impact

plt.text((expRecord.timespan.begin - TimeDelta(36, format='sec')).isot, 0.40, f"RMS error = {m2HexRms:.2f} arcsec")
saveFilename = f"/home/c/cslage/u/MTMount/mount_plots/Hexapod_Errors_{expId}.png"
plt.savefig(saveFilename)
